In [1]:
import re
import math

WORDS = "../words2.txt"
CROSSWORD = "../crossword2.txt"

In [2]:
def read_words(path):
    f = open(path, "r")
    lines = f.readlines()
    return [line.strip() for line in lines]

words = read_words(WORDS)
print(words)

['Aarhus', 'Aaron', 'Ababa', 'aback', 'abaft', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandons', 'abase', 'abased', 'abasement', 'abasements', 'abases', 'abash', 'abashed', 'abashes', 'abashing', 'abasing', 'abate', 'abated', 'abatement', 'abatements', 'abater', 'abates', 'abating', 'Abba', 'abbe', 'abbey', 'abbeys', 'abbot', 'abbots', 'Abbott', 'abbreviate', 'abbreviated', 'abbreviates', 'abbreviating', 'abbreviation', 'abbreviations', 'Abby', 'abdomen', 'abdomens', 'abdominal', 'abduct', 'abducted', 'abduction', 'abductions', 'abductor', 'abductors', 'abducts', 'Abe', 'abed', 'Abel', 'Abelian', 'Abelson', 'Aberdeen', 'Abernathy', 'aberrant', 'aberration', 'aberrations', 'abet', 'abets', 'abetted', 'abetter', 'abetting', 'abeyance', 'abhor', 'abhorred', 'abhorrent', 'abhorrer', 'abhorring', 'abhors', 'abide', 'abided', 'abides', 'abiding', 'Abidjan', 'Abigail', 'Abilene', 'abilities', 'ability', 'abject', 'abjection', 'abjections', 'abjectly', 'abjectness', 'abjure', 'a

In [3]:
hashes_regex = re.compile("\.+")

class Line:
    def __init__(self, origin, end):
        self.origin = origin
        self.end = end
    def __repr__(self):
        return "<origin: {}, end: {}>".format(str(self.origin), str(self.end))
    def length(self):
        return max(math.fabs(self.origin[0] - self.end[0]), math.fabs(self.origin[1] - self.end[1]))+1
    def get_type(self):
        if self.is_vertical():
            return "vertical"
        if self.is_horizontal():
            return "horizontal"
    def is_vertical(self):
        return self.origin[0]==self.end[0] and self.origin[1]!=self.origin[1]
    def is_horizontal(self):
        return self.origin[1]==self.end[1] and self.origin[0]!=self.origin[0]
    

def parse_horizontally(path):
    f = open(path, "r")
    horizontal_dots = []
    lines = f.readlines()
    for i, line in enumerate(lines):
        line = line.strip()
        positions = hashes_regex.finditer(line)
        for pos in positions:
            dot_line = Line((pos.span()[0], i), (pos.span()[1]-1, i))
            if dot_line.length()>1:
                horizontal_dots.append(dot_line)
    return horizontal_dots

def get_columns(path):
    f = open(path, "r")
    lines = f.readlines()
    columns = ['' for i in range(len(lines))]
    for line in lines:
        for caract in enumerate(line):
            if caract[1] != "\n":
                columns[caract[0]] += caract[1]
    return columns
    
# NB : the y axis is reversed
def parse_vertically(path):
    vertical_dots = []
    columns = get_columns(path)
    for i, column in enumerate(columns):
        positions = hashes_regex.finditer(column)
        for pos in positions:
            dot_column = Line((i, pos.span()[0]), (i, pos.span()[1]-1))
            if dot_column.length()>1:
                vertical_dots.append(dot_column)
    return vertical_dots

def read_crossword(path):
    # Horizontal lines
    horizontal_lines = parse_horizontally(path)
    # Vertical lines
    vertical_lines = parse_vertically(path)
    return horizontal_lines, vertical_lines
    
hlines, vlines = read_crossword(CROSSWORD)
print(hlines)
print(vlines)

[<origin: (3, 1), end: (12, 1)>, <origin: (1, 3), end: (7, 3)>, <origin: (9, 3), end: (13, 3)>, <origin: (1, 5), end: (4, 5)>, <origin: (6, 5), end: (13, 5)>, <origin: (1, 7), end: (6, 7)>, <origin: (8, 7), end: (13, 7)>, <origin: (1, 9), end: (8, 9)>, <origin: (10, 9), end: (13, 9)>, <origin: (1, 11), end: (5, 11)>, <origin: (7, 11), end: (13, 11)>, <origin: (2, 13), end: (11, 13)>]
[<origin: (1, 2), end: (1, 11)>, <origin: (3, 1), end: (3, 5)>, <origin: (3, 7), end: (3, 13)>, <origin: (5, 1), end: (5, 4)>, <origin: (5, 6), end: (5, 13)>, <origin: (7, 1), end: (7, 6)>, <origin: (7, 8), end: (7, 13)>, <origin: (9, 1), end: (9, 8)>, <origin: (9, 10), end: (9, 13)>, <origin: (11, 1), end: (11, 7)>, <origin: (11, 9), end: (11, 13)>, <origin: (13, 3), end: (13, 12)>]


In [4]:
def find_intersection(h_line, v_line):
    # Better: use the fact that it is necessary for the intersection to have v_line x position and h_line y position
    a = h_line.origin[0]<= v_line.origin[0]
    b = v_line.origin[0]<=h_line.end[0]
    c = v_line.origin[1]<=h_line.origin[1]
    d = v_line.origin[1]<= h_line.origin[1]
    e = h_line.origin[1]<=v_line.end[1]
    f = v_line.origin[0]>=h_line.origin[0]
    if a and b and c and d and e and f:
        return (v_line.origin[0], h_line.origin[1])
    else:
        return None

def find_intersections(hlines, vlines):
    intersections = []
    """
    [
        {
            "which": (
                nieme ligne de dots hlines,
                mieme colonne de dots vlines
                ),
            "position": (x, y)
    ]
    """
    for i, h_line in enumerate(hlines):
        for j, v_line in enumerate(vlines):
            intersection = find_intersection(h_line, v_line)
            if intersection != None:
                u = intersection[0] - h_line.origin[0]
                v = intersection[1] - v_line.origin[1]
                intersections.append({"position_in_crossword":intersection, "position_in_lines":(u, v), "which":(i, j)})
    return intersections

intersections = find_intersections(hlines, vlines)
print(intersections)

[{'position_in_crossword': (3, 1), 'position_in_lines': (0, 0), 'which': (0, 1)}, {'position_in_crossword': (5, 1), 'position_in_lines': (2, 0), 'which': (0, 3)}, {'position_in_crossword': (7, 1), 'position_in_lines': (4, 0), 'which': (0, 5)}, {'position_in_crossword': (9, 1), 'position_in_lines': (6, 0), 'which': (0, 7)}, {'position_in_crossword': (11, 1), 'position_in_lines': (8, 0), 'which': (0, 9)}, {'position_in_crossword': (1, 3), 'position_in_lines': (0, 1), 'which': (1, 0)}, {'position_in_crossword': (3, 3), 'position_in_lines': (2, 2), 'which': (1, 1)}, {'position_in_crossword': (5, 3), 'position_in_lines': (4, 2), 'which': (1, 3)}, {'position_in_crossword': (7, 3), 'position_in_lines': (6, 2), 'which': (1, 5)}, {'position_in_crossword': (9, 3), 'position_in_lines': (0, 2), 'which': (2, 7)}, {'position_in_crossword': (11, 3), 'position_in_lines': (2, 2), 'which': (2, 9)}, {'position_in_crossword': (13, 3), 'position_in_lines': (4, 0), 'which': (2, 11)}, {'position_in_crossword

In [5]:
class Crossword:
    def __init__(self, path):
        self.path = path
        self.find_lines()
        self.find_intersections()
    def find_lines(self):
        self.hlines, self.vlines = read_crossword(self.path)
    def find_intersections(self):
        self.intersections = find_intersections(self.hlines, self.vlines)
    def __repr__(self):
        return "horizontal lines: {}\nvertical lines: {}\nintersections: {}>".format(self.hlines, self.vlines, self.intersections)
        
cw = Crossword(CROSSWORD)
print(cw)

horizontal lines: [<origin: (3, 1), end: (12, 1)>, <origin: (1, 3), end: (7, 3)>, <origin: (9, 3), end: (13, 3)>, <origin: (1, 5), end: (4, 5)>, <origin: (6, 5), end: (13, 5)>, <origin: (1, 7), end: (6, 7)>, <origin: (8, 7), end: (13, 7)>, <origin: (1, 9), end: (8, 9)>, <origin: (10, 9), end: (13, 9)>, <origin: (1, 11), end: (5, 11)>, <origin: (7, 11), end: (13, 11)>, <origin: (2, 13), end: (11, 13)>]
vertical lines: [<origin: (1, 2), end: (1, 11)>, <origin: (3, 1), end: (3, 5)>, <origin: (3, 7), end: (3, 13)>, <origin: (5, 1), end: (5, 4)>, <origin: (5, 6), end: (5, 13)>, <origin: (7, 1), end: (7, 6)>, <origin: (7, 8), end: (7, 13)>, <origin: (9, 1), end: (9, 8)>, <origin: (9, 10), end: (9, 13)>, <origin: (11, 1), end: (11, 7)>, <origin: (11, 9), end: (11, 13)>, <origin: (13, 3), end: (13, 12)>]
intersections: [{'position_in_crossword': (3, 1), 'position_in_lines': (0, 0), 'which': (0, 1)}, {'position_in_crossword': (5, 1), 'position_in_lines': (2, 0), 'which': (0, 3)}, {'position_in_

In [6]:
from constraint_programming import constraint_programming

In [7]:
class CrosswordProblem:
    def __init__(self, wordspath, crosswordspath):
        self.words = read_words(wordspath)
        self.crossword = Crossword(crosswordspath)
    def __repr__(self):
        return "Words: {}\nCrossword:\n{}".format(self.words, self.crossword)

pb = CrosswordProblem(WORDS, CROSSWORD)
pb

Words: ['Aarhus', 'Aaron', 'Ababa', 'aback', 'abaft', 'abandon', 'abandoned', 'abandoning', 'abandonment', 'abandons', 'abase', 'abased', 'abasement', 'abasements', 'abases', 'abash', 'abashed', 'abashes', 'abashing', 'abasing', 'abate', 'abated', 'abatement', 'abatements', 'abater', 'abates', 'abating', 'Abba', 'abbe', 'abbey', 'abbeys', 'abbot', 'abbots', 'Abbott', 'abbreviate', 'abbreviated', 'abbreviates', 'abbreviating', 'abbreviation', 'abbreviations', 'Abby', 'abdomen', 'abdomens', 'abdominal', 'abduct', 'abducted', 'abduction', 'abductions', 'abductor', 'abductors', 'abducts', 'Abe', 'abed', 'Abel', 'Abelian', 'Abelson', 'Aberdeen', 'Abernathy', 'aberrant', 'aberration', 'aberrations', 'abet', 'abets', 'abetted', 'abetter', 'abetting', 'abeyance', 'abhor', 'abhorred', 'abhorrent', 'abhorrer', 'abhorring', 'abhors', 'abide', 'abided', 'abides', 'abiding', 'Abidjan', 'Abigail', 'Abilene', 'abilities', 'ability', 'abject', 'abjection', 'abjections', 'abjectly', 'abjectness', 'abju

In [8]:
# A line and its word have the same length
var = {str(line): set([word for word in words if len(word)==line.length()]) for line in cw.hlines+cw.vlines}
print(var)
P = constraint_programming(var)


{'<origin: (3, 1), end: (12, 1)>': {'bedspreads', 'fragmented', 'preeminent', 'diversions', 'hitchhiker', 'noticeably', 'repulsions', 'conserving', 'postscript', 'Prokofieff', 'regulators', 'resistable', 'architects', 'councillor', 'resultants', 'villainous', 'compliance', 'nationhood', 'glistening', 'enthusiast', 'attainably', 'restlessly', 'translated', 'characters', 'Appaloosas', 'appointive', 'superminis', 'vocational', 'Oconomowoc', 'freshening', 'savageness', 'invalidate', 'redundancy', 'separation', 'ciphertext', 'spotlessly', 'blanketers', 'asteroidal', 'deliveries', 'Belshazzar', 'guaranteed', 'properties', 'recognizes', 'chimpanzee', 'indirectly', 'hysteresis', 'Christlike', 'littleness', 'polytheist', 'modularity', 'compressor', 'Englanders', 'correlates', 'Sanderling', 'anchorages', 'Macdougall', 'horseshoer', 'derivative', 'instructed', 'originator', 'Montpelier', 'hypodermic', 'childbirth', 'complicate', 'Berkshires', 'transiency', 'domination', 'weathering', 'adulterous'

In [ ]:
# Intersections
def get_intersection_possibilities(i, j):
    return {(word, other_word) for word in words for other_word in words if (word[i]==other_word[j] and word!=other_word)}

for intersection in intersections:
    which_h = intersection['which'][0]
    hline = hlines[which_h]
    which_v = intersection['which'][1]
    vline = vlines[which_v]
    SAME_LETTER_INTERSECT = get_intersection_possibilities(*intersection['position_in_lines'])
    P.addConstraint(str(hline), str(vline), SAME_LETTER_INTERSECT)


In [37]:
lines = cw.hlines + cw.vlines

# A word is used only once
NEQ = {(i, j) for i in words for j in words if i!=j}
for line in lines:
    for other_line in lines:
        if line!=other_line:
            P.addConstraint(str(line), str(other_line), NEQ)

{'<origin: (1, 2), end: (3, 2)>': 'aaa', '<origin: (2, 1), end: (2, 3)>': 'aab'}


In [ ]:
sol = P.solve()
print(sol)